## Notebook for event GW190521
This notebook loads the posterior points and prior provided by [GWOSC](https://gwosc.org/), and (re)constructs the likelihood using [bilby](https://bilby-dev.github.io/bilby/)

The event GW190521 has a primary merger BH mass that lies within the pair-instability mass gap (see e.g. [2009.01190](https://arxiv.org/abs/2009.01190) and [2009.01075](https://arxiv.org/abs/2009.01075))

### Imports

In [ ]:
# General purpose
import numpy as np
import matplotlib.pyplot as plt
import os
import subprocess

# Bilby
import bilby
from bilby.gw.detector import InterferometerList
from bilby.gw.likelihood import GravitationalWaveTransient
from bilby.gw.waveform_generator import WaveformGenerator
from bilby.gw.source import lal_binary_black_hole
logger = bilby.core.utils.logger

# GWOSC
from gwosc.datasets import event_gps
from gwosc.api import fetch_event_json
from gwosc.locate import get_event_urls
from gwosc import datasets

# Getdist
from getdist import MCSamples
from getdist import plots

# Other
from gwpy.timeseries import TimeSeries
from pesummary.gw.fetch import fetch_open_samples

EVENT = "GW151226"

### Fetching event posterior samples

In [ ]:
# Check if directory exists. If not, create it
if not os.path.exists(f"{EVENT}"):
    os.mkdir(f"{EVENT}")

# Event is in the GWTC-1 catalog, so we can download it as-is
data = fetch_open_samples(EVENT, 
                          # catalog="GWTC-2", 
                          # unpack=True, 
                          read_file=True,
                          delete_on_exit=False, 
                          outdir=f"./{EVENT}", 
                          path=f"{EVENT}.h5")

# Samples and prior information
samples = data.samples_dict
priors = data.priors

calibration = priors["calibration"]
sample_prior = priors["samples"]

### MCSamples
This event contains multiple waveform chains. I have extracted only "PublicationSamples" here, but in principle the following are available:
* C01:IMRPhenomPv3HM
* C01:NRSur7dq4
* C01:SE0BNRv4PHM
* PrecessingSpinIMRHM
* PublicationSamples

In [ ]:
print(samples.keys())

In [ ]:
# MODEL = "PublicationSamples"
# MODEL = "C01:IMRPhenomPv3HM"
# MODEL = "C01:NRSur7dq4"
MODEL = "C01:IMRPhenomXPHM"

# MCSamples object
mcs_object = MCSamples(samples = samples[MODEL].samples.T, names = samples[MODEL].keys())

### Likelihood
Setting up the likelihood consists of 3 parts:
1) Creating a bilby interferometer object containing time-domain and PSD data for each detector
2) Setting up the priors in a bilby prior object
3) Setting up the model

#### 1. Interferometer objects

In [ ]:
# Timings info
trigger_time = datasets.event_gps(EVENT)

duration = 4  # Analysis segment duration
post_trigger_duration = 2  # Time between trigger time and end of segment
end_time = trigger_time + post_trigger_duration
start_time = end_time - duration

# Detector frequency info
maximum_frequency = 512
minimum_frequency = 20

# Detectors available as metadata
detectors = calibration[MODEL].detectors

# PSD data already available
PSD_all = data.psd[MODEL]
ifo_list = InterferometerList([])

for det in detectors:
    # Fetch time-domain data
    time_data = TimeSeries.fetch_open_data(det, start_time, end_time)
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    ifo.strain_data.set_from_gwpy_timeseries(time_data)

    # Add PSD
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=PSD_all[det].frequencies, psd_array=PSD_all[det].strains)

    ifo.maximum_frequency = maximum_frequency
    ifo.minimum_frequency = minimum_frequency
    ifo_list.append(ifo)

#### 2. Priors

In [ ]:
# These are sampled from the prior. I guess we can reconstruct the functional form
# using e.g. KDE?
prior_samples = sample_prior[MODEL]

# Just use default priors??
bilby_priors = bilby.gw.prior.BBHPriorDict()

#### 3. Model

In [ ]:
# Create a waveform generator (uses lal model
waveform_generator = bilby.gw.WaveformGenerator(
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments={
        "waveform_approximant": MODEL,
        "reference_frequency": 50,
    },
)

#### 4. Likelhood

In [ ]:
# Just a standard likelihood given our data and priors
likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    ifo_list,
    waveform_generator,
    priors=bilby_priors,
    time_marginalization=True,
    phase_marginalization=False,
    distance_marginalization=True,
)